In [1]:
import numpy as np

np.random.seed(42) # garante reprodutibilidade aos geradores de números aleatórios

import pandas as pd
from sklearn.datasets import fetch_california_housing # importa dataset "desconstruído"

california_housing = fetch_california_housing() # atribuí dataset "desconstruído" à um dicionário

# Controi as fetures em um DataFrame
california_housing_df = pd.DataFrame(california_housing["data"], columns = california_housing["feature_names"])
# Uni ao DataFrame o target
california_housing_df["MedHouseVal"] = california_housing["target"]

# Divide o DataFrame em features (x) e target (y)
X = california_housing_df.drop("MedHouseVal", axis = 1)
y = california_housing_df["MedHouseVal"]

from sklearn.model_selection import train_test_split 

# Divide os dados (X e y) em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor() # intancia modelo "Random Forest" com hiper-parâmetros padrões

model.fit(X_train, y_train) # treina o modelo com dados de treino

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


# Como Avaliar Modelos/Estimators de Regressão?

* **R² Score**

* **Mean Absulute Error (MAE)**

* **Mean Square Error (MSE)**

Para mais informações, veja a [documentação](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics) completa.

## 1. R² Score

O `R²`, também chamado de **Coeficiente de Determinação**, é uma métrica usada para avaliar o desempenho de **modelos de regressão**. Ele indica quanto da variabilidade dos dados é explicada pelo modelo, em termos simples: **mede o quão bem os valores previstos se ajustam aos valores reais**.

### Interpretação da métrica

* **R² = 1.0** → predição **perfeita** (todos os pontos previstos estão exatamente nos valores reais)

* **R² = 0.0** → o modelo não explica **nada além da média** (equivalente a um modelo que só chutaria a média dos dados)

* **R² < 0.0** → modelo **pior que um chute da média**

### Na prática, temos:

| Faixa de R²          | Avaliação típica (mas depende do contexto) |
| -------------------- | ------------------------------------------ |
| **R² ≥ 0.90**        | Excelente                                  |
| **0.75 ≤ R² < 0.90** | Muito bom                                  |
| **0.50 ≤ R² < 0.75** | Razoável / aceitável                       |
| **R² < 0.50**        | Fraco / modelo pode estar subajustado      |
| **R² < 0.0**         | Modelo pior que previsão da média          |


In [2]:
print(f".score() retorna métrica padão, neste caso R²: {model.score(X_test, y_test)}")

.score() retorna métrica padão, neste caso R²: 0.806652667101436


In [3]:
from sklearn.metrics import r2_score #  importa a métrica R² (coeficiente de determinação) Score

# Cria um modelo "aleatório"
# Modelo extremamente simples que só prevê a média de "y_test" para todos os pontos -  pior desempenho "aceitável"
y_test_mean = np.full(len(y_test), y_test.mean())

# Armazena a avaliação, com R² score, do modelo "aleatório"
result1 = r2_score(
    y_true = y_test,
    y_pred = y_test_mean
)

# Armazena a avaliação, com R² score, do modelo "perfeito"
result2 = r2_score(
    y_true = y_test,
    y_pred = y_test
)

y_preds = model.predict(X_test) # Previsões reais feitas pelo seu modelo treinado

# Armazena a avaliação, com R² score, do modelo real
result3 = r2_score(
    y_true = y_test,
    y_pred = y_preds
)

# Impressão clara dos 3 cenários: baseline "aleatório", modelo "perfeito", modelo real
print(f'r2_score() retorne o R² do modelo "aleatório": {result1}')
print(f'r2_score() retorne o R² do modelo "prefito": {result2}')
print(f'r2_score() retorne o R² do modelo real: {result3}')

r2_score() retorne o R² do modelo "aleatório": 0.0
r2_score() retorne o R² do modelo "prefito": 1.0
r2_score() retorne o R² do modelo real: 0.806652667101436


## 2. Mean Absolute Error (MAE)

O `MAE` (**Erro Absoluto Médio**) é uma métrica que mede, em média, quanto o modelo erra em relação aos valores reais — sem considerar o sinal do erro (positivo ou negativo), em outros termos:  **mostra o tamanho médio dos erros entre as previsões e os valores reais, em unidades absolutas**.

### Na práticas, temos:

Se o `MAE` for, por exemplo, **12.3**, isso significa que:

* O modelo, em média, erra **12.3 unidades** em relação ao valor real; e

* A unidade do erro é a mesma da variável de saída (ex: R$, kg, °C, etc).


In [4]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(
    y_test,
    y_preds
)

print(f"MAE: {mae}")

MAE: 0.32656738464147306


In [5]:
# Cria DataFrame p/ melhor visualização
# Compara valores reais com valores preditos
comparative_df = pd.DataFrame(data={
    "valores_reais": y_test,
    "valores_preditos": y_preds
})

# Cria um novo atributo (diferença de ambos os valores)
comparative_df["diferenca"] = comparative_df["valores_reais"] - comparative_df["valores_preditos"]

# Exibe as 5 primeiras linhas do DataFrame
comparative_df.head()

,valores_reais,valores_preditos,diferenca
20046,0.47700,0.493900,-0.016900
3024,0.45800,0.754940,-0.296940
15663,5.00001,4.928596,0.071414
20484,2.18600,2.540240,-0.354240
9814,2.78000,2.331760,0.448240


In [6]:
# Faz a média absoluta (anula positivo e negativo) das diferenças entre os valores reais e prefitos de y
print(f'Média absoluta da coluna "diferenca": {np.abs(comparative_df["diferenca"]).mean()}')
# (re)Imprime valor de MAE anteriormente calculado
print(f"MAE: {mae}")

Média absoluta da coluna "diferenca": 0.32656738464147306
MAE: 0.32656738464147306


## 3. Mean Squared Error (MSE)

O `MSE` (**Erro Quadrático Médio**) é uma métrica usada para avaliar modelos de regressão. Ele mede o erro médio entre as previsões e os valores reais, elevando esse erro ao quadrado, em outros termos: **mostra o quanto o modelo está errando, mas penaliza mais fortemente erros grandes** (por causa da elevação ao quadrado).

### Como calcular no SciKir-Learn

<pre><code>
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)

print(f'MSE: {mse:.2f}')
</code></pre>

### Exemplo prátcio

| Valor real | Valor previsto | Erro | Erro² |
| ---------- | -------------- | ---- | ----- |
| 100        | 90             | -10  | 100   |
| 150        | 170            | +20  | 400   |
| 130        | 125            | -5   | 25    |

<br>

$$
\text{MSE} = \frac{100 + 400 + 25}{3} = \frac{525}{3} = 175.00
$$

<br>

O MSE retorna o erro médio ao quadrado, o que significa que **erros grandes têm impacto desproporcional**.

### ⚠️ Limitações

* Por estar em unidades ao quadrado, o MSE não é tão intuitivo quanto o MAE.

* É muito sensível a *outliers*: um único erro muito grande pode distorcer bastante o resultado.

---

## Comparativo entre `MAE` e `MSE`

| Critério                 | **MAE (Mean Absolute Error)**               | **MSE (Mean Squared Error)**                        |
| ------------------------ | ------------------------------------------- | --------------------------------------------------- |
| Fórmula                  | Média dos valores absolutos dos erros       | Média dos erros ao quadrado                         |
| Penaliza erros grandes?  | ❌ Pouco                                     | ✅ Muito (erro grande é amplificado)                 |
| Sensível a outliers?     | ❌ Pouco sensível                            | ✅ Bastante sensível                                 |
| Unidade                  | Mesma do alvo (ex: R\$, kg)                 | Unidade ao quadrado (ex: R\$², kg²)                 |
| Interpretação            | Intuitiva: erro médio direto                | Menos intuitiva, mas usada internamente por modelos |
| Ideal para               | Quando você quer entender o erro médio real | Quando você quer punir fortemente os erros grandes  |
| Usado em algoritmos como | Modelos baseados em mediana                 | Regressão Linear, Redes Neurais, etc                |
